In [8]:
import pickle
import os
import sys
import nibabel as nib
import stitchSurfs as stitch
import subprocess

import projectUtils as prjUtils

sys.path.append('/host/verges/tank/data/daniel/00_commonUtils/00_code/genUtils/')
import gen

In [13]:
# equivolumetric surface generation
# options:
# LayNii: Takes cortical ribbon segmentation as input 
#   https://github.com/layerfMRI/LAYNII
# cortPro: takes volumes as input (generates pial, white surfaces)
def equiVolSurfs_LayNII(pial, white, n_surfs, out_prefix, laynii_pth = "/host/verges/tank/data/daniel/software/LayNii_v2.9.0"):
    # LayNii: https://github.com/layerfMRI/LAYNII
    # Citation: Huber, L., Poser, B. A., Bandettini, P. A., Arora, K., Wagstyl, K., Cho, S., Goense, J., Nothnagel, N., Morgan, A. T., van den Hurk, J., Mueller A. K., Reynolds, R. C., Glen, D. R., Goebel, R. W., Gulban, O. F. (2021). LayNii: A software suite for layer-fMRI. NeuroImage, 118091. https://doi.org/10.1016/j.neuroimage.2021.118091
    cmd = ""
    subprocess.run(cmd, shell=True)
    return save_names


def mp_MPC(id:str, ses:str, out_pth:str, bids_dir:str, microStrucImg_pth:str, ):
    
    cmd = "micapipe " \
    f"-sub {gen.fmt_id_ses(id, ses)} " \
    f"-out {out_pth} " \
    f"-bids {bids_dir} " \
    f"-MPC -microstructural_img {microStrucImg_pth}"
    subprocess.run(cmd, shell=True)
    return


def runCortPro(id:str, ses:str, dir_surfsIn:str, dir_out:str, n_surfs:int, naming_ptrn:str):
    prjUtils.make_dir(dir_out)
    
    fs_dir = "/data/mica1/01_programs/freesurfer-7.3.2" # Path to the FreeSurfer directory [required] (should contain standard license file, 'license.txt')"
    sing_dir  = "/data/mica1/01_programs/singularity/" # Path to the directory with singularities [required] (must contain micapipe-v0.2.3.simg and, if Freesurfer output is not yet available, fastsurfer_gpu.sif)"
    cortPro_pth = "/host/verges/tank/data/daniel/software/CortPro-main/"

    # subjects-dir: Path to Freesurfer-style SUBJECTS_DIR [required] (if the directory doesn't contain surfaces for the specified subject, Fastsurfer will be run)
    cmd = f"{cortPro_pth}/microstructure_profiling.sh --subject-id {gen.fmt_id_ses(id, ses)}" \
    f" --subjects-dir {dir_surfsIn}" \
    f" --output-dir {dir_out}" \
    f" --fs-dir {fs_dir}" \
    f" --sing-dir {sing_dir}" \
    f" --num-surfaces {n_surfs}" \
    f" --surface-output {naming_ptrn}" \
    f" --keep-inter-files" # optional

    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"Error running command: {cmd}")
        print(f"Return code: {result.returncode}")
        print(f"STDOUT: {result.stdout}")
        print(f"STDERR: {result.stderr}")
    else:
        print(f"Command executed successfully: {cmd}")
        if result.stdout:
            print(f"STDOUT: {result.stdout}")
    return

In [ ]:
runCortPro(id="PNC021", ses="01", 
           dir_surfsIn="/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/", 
           dir_out="/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/", 
           n_surfs=5, 
           naming_ptrn="stitched")

Error running command: /host/verges/tank/data/daniel/software/CortPro-main//microstructure_profiling.sh --subject-id sub-PNC021_ses-01 --subjects-dir /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/ --output-dir /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/ --fs-dir /data/mica1/01_programs/freesurfer-7.3.2 --sing-dir /data/mica1/01_programs/singularity/ --num-surfaces 5 --surface-output stitched --keep-inter-files
Return code: 1
STDOUT: [ERROR] micapipe-v0.2.3.simg not found at: /data/mica1/01_programs/singularity/

STDERR: 
